In [928]:
## L SHAPED METHOD Q1 and Q2

In [929]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum as qsum
from itertools import product
import pandas as pd
from pandas import DataFrame
import numpy as np

In [930]:
originalCapacity = [20, 22, 17, 19, 18] # Plant capacity in thousands of units
capacityMultiplier = 2.5
capacity = capacityMultiplier*np.array(originalCapacity)

In [931]:
# Warehouse demand in thousands of units
demand = [15, 18, 14, 20]

# Fixed costs for each plant
fixedCosts = [12000, 15000, 17000, 13000, 16000]

# Transportation costs per thousand units
transCosts = [[4000, 2000, 3000, 2500, 4500],
              [2500, 2600, 3400, 3000, 4000],
              [1200, 1800, 2600, 4100, 3000],
              [2200, 2600, 3100, 3700, 3200]]

maxTransCost = max([max(i) for i in transCosts])
emergencyTransportMultiplier = 3
emergencyTransportCost = emergencyTransportMultiplier*maxTransCost

In [932]:
plants = range(len(capacity))
warehouses = range(len(demand))

In [933]:
facStatusCombs = list(product([0,1], repeat=len(plants)))
xi = {i : facStatusCombs[i] 
      for i in range(len(facStatusCombs))}
scenarios = list(xi.keys())

probFacilityFailure = 0.1

def getProbOfCombin(combination):
    prod = 1
    for i in range(len(combination)):
        if combination[i] == 0:
            prod *= probFacilityFailure
        elif combination[i] == 1:
            prod *= 1 - probFacilityFailure
    return prod

probOfScenario = {i : getProbOfCombin(facStatusCombs[i] ) 
      for i in range(len(facStatusCombs))}

In [934]:
def SecondStageFunction(x,xi):
    m1 = gp.Model('Second_Stage')
    y = m1.addVars(warehouses, plants, 
                      name="transportation")

    e = m1.addVars(warehouses, 
                      name="excessTransportation")
    capacity_constraints = m1.addConstrs(
    (y.sum('*', p) <= capacity[p] * xi[p] * x[p] for p in plants),
    name="Capacity"
    )

    demand_constraints = m1.addConstrs(
        (y.sum(w, '*') + e[w] == demand[w] for w in warehouses),
        name="Demand")
    dual_values_capacity = {}
    dual_values_demand = {}
    m1.setObjective((qsum(transCosts[w][p]*y[w,p] 
                                        for p in plants 
                                        for w in warehouses) 
                    + qsum(emergencyTransportCost*e[w] 
                           for w in warehouses)), GRB.MINIMIZE)
    
    m1.Params.LogToConsole = 0
    m1.optimize()
    
    dual_values_capacity = {}
    rhs_values_capacity = {}
    
    dual_values_demand = {}
    rhs_values_demand = {}
    
    for p in plants:
        constrName = f"Capacity[{p}]"
        dual_values_capacity[p] = m1.getConstrByName(constrName).Pi
        #rhs_values_capacity[p] = constraint.rhs

    for w in warehouses:
        constrName = f"Demand[{w}]"
        dual_values_demand[w] = m1.getConstrByName(constrName).Pi
        #rhs_values_demand[p] = constraint.rhs
    duals = {0: dual_values_capacity, 1:  dual_values_demand}
    
    return m1.ObjVal, duals #_values_capacity, dual_values_demand#, rhs_values_capacity, rhs_values_demand

In [935]:
#xSoln = {name : x[name].x for name in x}
def getExpectedSecondStageObjective(a,xi):
    expectedObjVal = sum(probOfScenario[s]*secondstageFunction(a,xi[s])[0] for s in scenarios)
    return expectedObjVal

In [936]:
def getInterceptAndSlopesForScenario(duals,s):
    rhs_p = {}
    rhs_w = {}
    
    T_p = {}
    T_w = {}
    
    slope_p = {}
    slope_w = {}
    
    for p in plants:
        rhs_p[p] = 0 #- originalCapacity[p]*xi[s][p]
        T_p[p] = - capacity[p]*xi[s][p]
            
    for w in warehouses:
        rhs_w[w] = demand[w]
        T_w[w] = 0
    #duals_p = secondstageFunction(x,xi[s])[1]
    #duals_w = secondstageFunction(x,xi[s])[2]
    #rhs_p = secondstageFunction()[3]
    #rhs_w = secondstageFunction()[4]
    for p in plants:
        slope_p[p] = duals[0][p]*T_p[p]
    for w in warehouses:
        slope_w[w] = duals[1][w]*T_w[w]
        
    i1 = sum(duals[0][p]*rhs_p[p]
                         for p in plants)                    
    i2 = sum(duals[1][w]*rhs_w[w]
                         for w in warehouses)
    interceptValue = i1 + i2
    #slopes = {0: slope_p, 1: slope_w}
    return interceptValue, slope_p, slope_w
    

In [937]:
def getInterceptsAndSlopesLists(scenarios, dualsForScenarios):
    intsAndSlopesForScenarios = {s: (intercept, slope) for s in scenarios for intercept, slope, _ in [getInterceptAndSlopesForScenario(dualMult[s], s)]}

    
    intercepts = {s: round(intsAndSlopesForScenarios[s][0]) for s in scenarios}
    slopesLists = {s: [round(intsAndSlopesForScenarios[s][1][i]) for i in intsAndSlopesForScenarios[s][1]] for s in scenarios}
    
    # Create a DataFrame for visualization
    df = DataFrame.from_dict({s: {'Intercept': intercepts[s], 'Slopes': slopesLists[s]} for s in scenarios}, orient='index')
    #print(df)
    
    return intercepts, slopesLists

In [938]:
def addCut(intercepts, slopes):
    expectedSlopes = {}
    expectedIntercept = sum(probOfScenario[s]*intercepts[s] for s in scenarios)
    for p in plants:
        expectedSlopes[p] = sum(probOfScenario[s]*slopes[s][p] for s in scenarios) 
    cut = m.addConstr(theta >= expectedIntercept - qsum(x[p]*expectedSlopes[p] for p in plants) 
                    ,"BendersCut")
    print("Cut added:", theta >= expectedIntercept - x.prod(expectedSlopes))

#addCut(intercepts, slopes)

In [939]:
m = gp.Model("FacilityLocation")

x = m.addVars(plants,
                 vtype=GRB.BINARY,
                 name="x")
theta = m.addVar(name = "theta", lb = 0)
m.setObjective(x.prod(fixedCosts) + theta, GRB.MINIMIZE)
m.Params.LogToConsole = 0 
m.optimize()


In [940]:
print(xSoln)
#theta = {theta.}
theta_val = theta.X
print(theta_val)

{0: 1.0, 1: 1.0, 2: -0.0, 3: 1.0, 4: -0.0}
0.0


In [941]:
print(m.Objval)

0.0


In [942]:
A = 0
LB = 0
UB = 100000000000000000000
while (UB - LB) >= 0.01:
    dualMult = {}
    total_second_stage_cost = 0
    D = {}
    m.Params.LogToConsole = 0 
    m.optimize()
    xSoln = {name : x[name].x for name in x}
    theta_val = theta.X
    LB = theta_val
    
    for s in scenarios:
        second, dualMult[s] = SecondStageFunction(xSoln,xi[s])
        total_second_stage_cost += probOfScenario[s]*second
        D[s] = getInterceptAndSlopesForScenario(dualMult[s],s)
    
    #result = [fixedCosts[i]*xsoln[i] for i in range(len(fixedCosts))]
    #H = sum(result)
    UB = total_second_stage_cost
    #print(UB)
    i,s = getInterceptsAndSlopesLists(scenarios, D)
    A += 1
    print(f'Cut: {A}')
    cut = addCut(i,s) 
    print(' ')


Cut: 1
Cut added: <gurobi.TempConstr: <gurobi.LinExpr: theta> >= <gurobi.LinExpr: 904500.0000000002 + -553500.0000000001 x[0] + -579150.0000000001 x[1] + -416925.0000000001 x[2] + -470250.0000000001 x[3] + -425250.0000000001 x[4]>>
 
Cut: 2
Cut added: <gurobi.TempConstr: <gurobi.LinExpr: theta> >= <gurobi.LinExpr: 304056.0000000001 + -69750.0 x[0] + -129195.00000000003 x[1] + -77838.75000000001 x[2] + -41895.000000000015 x[3] + -78975.0 x[4]>>
 
Cut: 3
Cut added: <gurobi.TempConstr: <gurobi.LinExpr: theta> >= <gurobi.LinExpr: 150817.95000000004 + -6975.000000000002 x[0] + -2588.8500000000004 x[1] + -2117.520000000001 x[2] + -418.95000000000016 x[3] + -384.75000000000017 x[4]>>
 
Cut: 4
Cut added: <gurobi.TempConstr: <gurobi.LinExpr: theta> >= <gurobi.LinExpr: 282069.0000000001 + -53550.000000000015 x[0] + -53955.00000000001 x[1] + -76117.50000000003 x[2] + -87423.75000000003 x[3] + -78246.00000000003 x[4]>>
 
Cut: 5
Cut added: <gurobi.TempConstr: <gurobi.LinExpr: theta> >= <gurobi.LinE

In [943]:
print(A)

11


In [944]:
print(m.Objval)

184827.10000000003


In [945]:
print(xSoln)

{0: 1.0, 1: 1.0, 2: -0.0, 3: 1.0, 4: -0.0}


In [946]:
print(theta_val)

144827.10000000003


In [947]:
m2 = gp.Model("FacilityLocation")

x1 = m2.addVars(plants,
                 vtype=GRB.BINARY,
                 name="open")
theta1 = m2.addVars(scenarios, name = "theta", lb = 0)
m2.setObjective((x1.prod(fixedCosts)) + qsum(probOfScenario[s]*theta1[s] for s in scenarios), GRB.MINIMIZE)
m2.Params.LogToConsole = 0 
m2.optimize()

In [948]:
xSol = {name : x1[name].x for name in x}
print(xSol)
#theta = {theta.}
theta_va = qsum(probOfScenario[s] * theta1[s].X for s in scenarios).getValue()
print(theta_va)

{0: -0.0, 1: -0.0, 2: -0.0, 3: -0.0, 4: -0.0}
0.0


In [949]:
def Cut(intercepts, slopes,s):
    Slopes = {}
    Intercept = intercepts
    for p in plants:
        Slopes[p] = slopes[p] 
    cut = m2.addConstr(theta1[s] >= Intercept - qsum(x1[p]*Slopes[p] for p in plants) 
                    ,"BendersCut")
    cut_expr = " + ".join(f"{Slopes[p]}*x1[{p}]" for p in plants)
    print(f"Cut added for scenario {s}: theta1[{s}] >= {Intercept} - ({cut_expr})")

In [950]:
A = 0
LB1 = 0
UB1 = 100000000000000000000
while (UB1 - LB1) >= 0.0011:
    dualMult = {}
    total_second_stage_cost = 0
    D = {}
    m2.Params.LogToConsole = 0 
    m2.optimize()
    xSol = {name : x1[name].x for name in x}
    #print(xSoln)
    theta_va = qsum(probOfScenario[s] * theta1[s].X for s in scenarios).getValue()
    LB1 = theta_va
    A += 1
    for s in scenarios:
        second, dualMult[s] = SecondStageFunction(xSol,xi[s])
        total_second_stage_cost += probOfScenario[s]*second
        i,j, _ = getInterceptAndSlopesForScenario(dualMult[s],s)
        print(f'Cut in Iteration: {A}')
        cut = Cut(i,j,s)
        
    UB1 = total_second_stage_cost
    #print(UB1)
    #i,s = getInterceptsAndSlopesLists(scenarios, D)
    #cut = addCut(i,s) 

    #print(LB1)
    



Cut in Iteration: 1
Cut added for scenario 0: theta1[0] >= 904500.0 - (0.0*x1[0] + 0.0*x1[1] + 0.0*x1[2] + 0.0*x1[3] + 0.0*x1[4])
Cut in Iteration: 1
Cut added for scenario 1: theta1[1] >= 904500.0 - (0.0*x1[0] + 0.0*x1[1] + 0.0*x1[2] + 0.0*x1[3] + 472500.0*x1[4])
Cut in Iteration: 1
Cut added for scenario 2: theta1[2] >= 904500.0 - (0.0*x1[0] + 0.0*x1[1] + 0.0*x1[2] + 522500.0*x1[3] + 0.0*x1[4])
Cut in Iteration: 1
Cut added for scenario 3: theta1[3] >= 904500.0 - (0.0*x1[0] + 0.0*x1[1] + 0.0*x1[2] + 522500.0*x1[3] + 472500.0*x1[4])
Cut in Iteration: 1
Cut added for scenario 4: theta1[4] >= 904500.0 - (0.0*x1[0] + 0.0*x1[1] + 463250.0*x1[2] + 0.0*x1[3] + 0.0*x1[4])
Cut in Iteration: 1
Cut added for scenario 5: theta1[5] >= 904500.0 - (0.0*x1[0] + 0.0*x1[1] + 463250.0*x1[2] + 0.0*x1[3] + 472500.0*x1[4])
Cut in Iteration: 1
Cut added for scenario 6: theta1[6] >= 904500.0 - (0.0*x1[0] + 0.0*x1[1] + 463250.0*x1[2] + 522500.0*x1[3] + 0.0*x1[4])
Cut in Iteration: 1
Cut added for scenario 7:

In [951]:
print(m2.Objval)

184827.10000000006


In [952]:
print(A)

3
